<a href="https://colab.research.google.com/github/sparta313/Osintgram/blob/master/llam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch accelerate bitsandbytes

In [2]:
!apt-get install libstdc++6 -y
!apt --fix-broken install -y
!pip install --upgrade bitsandbytes
!pip install --upgrade transformers

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libstdc++6 is already the newest version (12.3.0-1ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "TheDrummer/Llama-3SOME-8B-v2"

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Загружаем модель (если мало VRAM, ставь load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")

print("✅ Модель загружена!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Модель загружена!


In [13]:
def chat_with_ai(prompt, max_length=250):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_length=max_length)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [14]:
user_input = "Describe a futuristic city at night with neon lights."
response = chat_with_ai(user_input)
print(response)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Describe a futuristic city at night with neon lights. The city is called Neo-Tokyo and it's a bustling metropolis filled with towering skyscrapers, holographic advertisements, and flying vehicles zipping through the air. The streets are crowded with people in futuristic clothing, some with cybernetic enhancements, as they go about their nightly routines.

The sky above is a deep indigo, with stars twinkling faintly through the haze of city lights. Neon signs in every color of the rainbow illuminate the towering skyscrapers, casting a kaleidoscope of hues across the pavement below. Holographic advertisements float through the air, their images shifting and morphing to catch the attention of passersby.

Flying vehicles, both personal and public, dart through the night sky, their headlights and taillights leaving trails of light as they weave between the towering structures. The vehicles range from sleek, high-tech hovercars to bulky cargo drones, all moving with a quiet hum.

On the stre

In [15]:
!pip install gradio
!pip install pyngrok
from pyngrok import ngrok
!ngrok config add-authtoken 2sblw4pecPruS0Dr5xwbfpdlcBH_6cJbFiQRsFjSa9y2dJDku
import gradio as gr

# Определяем функцию перед запуском интерфейса
def chat_with_ai(prompt, max_length=250):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_length=max_length)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

def chat_interface(message, history=[]):
    response = chat_with_ai(message)
    history.append(("Ты", message))
    history.append(("AI", response))
    return history, ""

with gr.Blocks() as demo:
    gr.Markdown("# 🤖 Llama-3SOME-8B-v2 Chatbot (Colab)")
    chatbot = gr.Chatbot(label="Чат с AI")
    user_input = gr.Textbox(label="Ваше сообщение")
    submit = gr.Button("Отправить")

    submit.click(chat_interface, inputs=[user_input, chatbot], outputs=[chatbot, user_input])

public_url = ngrok.connect(7860)
print("Gradio доступен по ссылке:", public_url)

demo.launch(share=True)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Gradio доступен по ссылке: NgrokTunnel: "https://d260-35-204-251-229.ngrok-free.app" -> "http://localhost:7860"
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6dcec4ca57f0967ee5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
